#Faceswap Colab
*Author: Sabut0l (Roman)*
    
 
 
Links for cooperation: 
 
VK:
https://vk.com/roman_kopylov
 
Github:
https://github.com/Sabut0l
 
Discord:
Sabut0l#4601

# Instructions

Google Colab Notebook for Faceswap
> **To use the program, rename the video files A and B to FaceA; FaceB. WITHOUT THIS, YOU WILL NOT GET DEEPFACE !**
 
Files you need to put on Google Drive
 
1.   face_a.zip
2.   face_b.zip
> Zipped files with extracted faces and alignments.fsa
3.   train.ini
> Your train.ini from your local system, in Windows this is in: C:\Users\YourName\faceswap\config
 
Here is what my filesystem looks like in Google Drive
*   /colab_files/faceswap/faces/face_a.zip
*   /colab_files/faceswap/faces/face_b.zip
*   /colab_files/faceswap/config/train.ini
 
Recommended folders to create on Google Drive:
*   /colab_files/faceswap/models/YourModelName
*   /colab_files/faceswap/output/timelapse
 
Here is what my filesystem looks like in this Google Colab after the "Mount Google Drive" step below.
*   /content/drive/My Drive/colab_files/faceswap/faces/face_a.zip
*   /content/drive/My Drive/colab_files/faceswap/faces/face_b.zip
*   /content/drive/My Drive/colab_files/faceswap/config/train.ini

# Usage Notes

This is my personal SOP for handling the Faceswap process. I find it is best to tweak your settings, extract, test your training, and convert everything using your local system.

But training on my laptop took weeks to get the same results that you can get from just days by using a Google Colab.

*   Extract, Align, then Zip all Faces A
*   Extract, Align, then Zip all Faces B
*   Modify the "Run Training" script below to your preferences
*   Train in this Colab Notebook
*   Watch Google Drive timestamp folder to see how the training is improving
*   Download YourModelName to your local system
*   Convert using the downloaded model


# Credits

Thank you Faceswap Community

*   https://faceswap.dev/
*   https://forum.faceswap.dev/
*   The other Github Google Colab notebooks which inspired this one



# Download videos for deepfake

In [ ]:
#@title Mount Google Drive
 
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#@title Download training data
!cp "/content/drive/My Drive/colab_files/faceswap/faces/face_a.zip" .
!cp "/content/drive/My Drive/colab_files/faceswap/faces/face_b.zip" .
 
!unzip face_a.zip -d face_a
!unzip face_b.zip -d face_b
 
!rm face_a.zip
!rm face_b.zip

When you want to keep faces exctract - it will help you

In [ ]:
#@title Save faces exctract
#=====SAVE FACES EXCTRACT=====
!zip -r face_a.zip "/content/face_a/"
!zip -r face_b.zip "/content/face_b/"
!cp "/content/face_a.zip" "/content/drive/My Drive/colab_files/faceswap/faces_extract/face_a.zip"
!cp "/content/face_b.zip" "/content/drive/My Drive/colab_files/faceswap/faces_extract/face_b.zip"
!rm -r /content/face_a
!rm -r /content/face_b

In [ ]:
#@title Import faces exctract
#=====IMPORT FACES EXCTRACT=====
!cp "/content/drive/My Drive/colab_files/faceswap/faces_extract/face_a.zip" .
!cp "/content/drive/My Drive/colab_files/faceswap/faces_extract/face_b.zip" .
!unzip face_a.zip -d /
!unzip face_b.zip -d /
!rm face_a.zip
!rm face_b.zip

# Setup Faceswap

In [ ]:
#@title Use this to check the assigned GPU
 
!cat /etc/os-release
 
def install_dependencies():
  !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi;
  !pip install gputil;
  !pip install psutil;
  !pip install humanize;%%capture
 
 
def printm():
 GPUs = GPU.getGPUs()
 
 if len(GPUs) == 0:
  print("No GPU available.")
  return
 
 gpu = GPUs[0]
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
 
from IPython.utils import io
from google.colab import drive
import psutil
import humanize
import os
 
with io.capture_output() as captured:
  install_dependencies()
print("Dependencies installed.")
 
 
import GPUtil as GPU
 
printm()

In [ ]:
#@title Set Time Zone
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/HST /etc/localtime
!date
 
#above is for HST, you can find yours in
#/usr/share/zoneinfo

In [ ]:
#@title Grab the latest Faceswap
!git clone https://github.com/deepfakes/faceswap.git

In [ ]:
#@title Copy configuration files
!cp "/content/drive/My Drive/colab_files/faceswap/config/train.ini" faceswap/config/
!ls -lA faceswap/config/
!cat faceswap/config/train.ini

In [ ]:
#@title Install Requirements

!pip install -r faceswap/requirements_nvidia.txt

In [ ]:
#@title Selecting a GPU in faceswap
!git clone https://github.com/Sabut0l/DeepFake_Faceswap_Colab
!cp "/content/DeepFake_Faceswap_Colab/utils.py" "/content/faceswap/lib/utils.py"
!rm -r "/content/DeepFake_Faceswap_Colab"

# Create deepface

**After you have completed the previous commands, you can start creating a *deepfake*** 

In [ ]:
#@title **FaceA Alignment**
!python faceswap/faceswap.py extract -i face_a/FaceA.mp4 -o face_a/extract_video

In [ ]:
#@title **FaceB Alignment**
!python faceswap/faceswap.py extract -i face_b/FaceB.mp4 -o face_b/extract_video

In [ ]:
#@title **Removing unnecessary faces**
#@markdown **If there are additional faces in your video, this command will help you remove them**
 
#@markdown **In the directory: /content/face_a/extract_video and /content/face_b/extract_video there are faces**
 
#@markdown **If there is an extra face, then at the end of the file name will be written _1, _2, _3, and so on. Find the excess and enter the line below. Without _**
 
import os, glob
 
name = "1" #@param {type:"string"}
 
directory = "/content/face_a/extract_video/" #@param ["/content/face_a/extract_video/", "/content/face_b/extract_video/"]
ch = "_" + name
file1 = directory + "*" + ch + ".png"
print(file1)
for file in glob.glob(file1):
    print("Delete - " + file)
    os.remove(file)

In [ ]:
 
#@title **Train**
#@markdown **Now have come to the longest stage - training**
#@markdown ---
#@markdown **Here's an example**
 
#@markdown ![example](https://forum.faceswap.dev/download/file.php?id=190&sid=2734a9aa2b52eb7f6dcac967eb9d2877) 
#set variables start
#@markdown ---
num_iterations = 100000 #@param {type:"integer"}
save_every =  360#@param {type:"integer"}
save_model_every = 25000 #@param {type:"integer"}
batch_num = "16"
num_gpus = "1"  
trainer_type = 'villain' #@param ['lightweight', 'original', 'iae', 'dfaker', 'unbalanced', 'villain', 'realface', 'dlight']
 
name_model_dir = "You_model" #@param {type:"string"}
#@markdown ---
#@markdown *Details about training types [HERE](https://forum.faceswap.dev/viewtopic.php?t=146)*
 
#@markdown The name of the model. Write in English letters. If you continue to training, remember the NAME! 
 
#@markdown You can see the stages of training in the directory: /content/time 
model_dir = "/content/drive/My Drive/colab_files/faceswap/models/" + name_model_dir
alignments_file_a = "/content/face_a/FaceA_alignments.fsa"
alignments_file_b = "/content/face_b/FaceB_alignments.fsa"
timelapse_dir = "/content/drive/My Drive/colab_files/faceswap/output/timelapse"
!python3 faceswap/faceswap.py train \
  -A 'face_a/extract_video' \
  -B 'face_b/extract_video' \
  -m '{model_dir}' \
  -t '{trainer_type}' \
  -bs '{batch_num}' \
  -it $num_iterations \
  -s $save_every \
  -ss $save_model_every \
  --timelapse-input-A '/content/face_a/extract_video' \
  --timelapse-input-B '/content/face_b/extract_video' \
  --timelapse-output '/content/time'

In [ ]:
#@title **Save Deepface in the format .mp4**
#@markdown **Choose a model for converting a Deepfake into a video**
name_model = "You_model" #@param {type:"string"}
#@markdown **Select the video that the "mask" will be applied to**
name_mp4 = "/content/face_a/FaceA.mp4" #@param ["/content/face_a/FaceA.mp4", "/content/face_b/FaceB.mp4"]
#@markdown **The video itself will be saved in Google drive in the directory: /content/drive/My Drive/colab_files/faceswap/outpute_dir_video**
model_v = "/content/drive/My Drive/colab_files/faceswap/models/" + name_model
!rm /content/drive/My Drive/colab_files/faceswap/outpute_dir_video/FaceA_converted.mp4
!python faceswap/faceswap.py convert -i '{name_mp4}' -o "/content/drive/My Drive/colab_files/faceswap/outpute_dir_video" -m '{model_v}' -w ffmpeg